# Reddit Scraping
This notebook uses PSAW to get archive comments retrieved by Pushishift immediately after they are created. This way, we can get comments before the risk of them being deleted.

Using this method, we can get older data, for instance, from 2021. **IMPORTANT**: we need to check that `created_utc` and `retrieved_utc` are close so we can ensure pushishift was retrieving posts immediately after creation at the chosen epoch.

___
### --- TO DO --- 
#### before we start scraping:
* decide on the epoch
* finalize dictionary of terms to use in the filter for PSAW
* decide on how to keep track of all the scraping instances. For instance, do we want to use name convention so every time the scraper makes a request and saves new DFs to Google Drive, each DF has a number in the name to reference the time or order in which it was scraped? For instance, comments_df_2022031214 or comments_df_001
* we will probably encounter problems with ratelimits and will have to learn more about it and trouble shoot it
___

In [269]:
# pip install psaw
# pip install praw

In [270]:
from psaw import PushshiftAPI
import reddit_auth
import praw
from datetime import datetime
import pandas as pd
from collections import Counter

In [195]:
import praw
import reddit_auth
reddit = praw.Reddit(
    client_id=reddit_auth.client_id,
    client_secret=reddit_auth.client_secret,
    user_agent=reddit_auth.user_agent, 
    password=reddit_auth.password,
    username=reddit_auth.username
)

# PSAW

### Get archive comments

In [406]:
# define epoch
after = int(datetime(2022,1,1,0,0).timestamp())
before = int(datetime(2022,1,2,0,0).timestamp())

# define our filter
our_list_of_terms = ['python','java']
our_filter = '|'.join(our_list_of_terms)

# define limit
our_limit = 10 # so testing can be faster

def get_historical_data(our_filter, our_limit, before, after):

    '''
    Input
    - our_filter: our list of terms. Needs to be separated by |
    - our_limit: number of comments to retrieve
    - before: upper time limit. Format: int(datetime(yyyy,m,d,h,m,s).timestamp())
    - after: lower time limit
    Output
    - comments_df
    
    Uses psaw to scrape historical data created between 'after' and 'before', 
    filtered by the list of terms in our_filter
    
    '''
    
    from psaw import PushshiftAPI
    api = PushshiftAPI()

    gen = api.search_comments(q=our_filter, 
                              limit=our_limit, 
                              after=after, 
                              before=before)
    comments_df = pd.DataFrame(gen)

    comments_df.drop(labels=['d_'], inplace=True, axis=1)
    comments_df['created_utc'] = pd.to_datetime(comments_df.created_utc, unit='s')
    comments_df['retrieved_utc'] = pd.to_datetime(comments_df.retrieved_utc, unit='s')
    
    return comments_df

Since comments are retrieved soon after being posted, their features are not up to date. To update them, we use praw.

# PRAW

#### Get updated information on each Comment
#### Get live data on each Submission (parent)
#### Get live data on each User

In [405]:
def get_live_data(df):
    '''
    Input
    - df: comments_df
    Output
    - comments_df
    - submissions_df
    - users_df
    
    Creates 2 new DFs:
    - submissions_df
    - users_df
    And updates df with current information
    
    Iterate over df to get identifying information and 
    use it to populate the 2 new DFs
    
    '''
    
    import praw
    import reddit_auth
    reddit = praw.Reddit(
        client_id=reddit_auth.client_id,
        client_secret=reddit_auth.client_secret,
        user_agent=reddit_auth.user_agent, 
        password=reddit_auth.password,
        username=reddit_auth.username
    )
    
    comments_df = df.copy()
    submissions_df = pd.DataFrame()
    users_df = pd.DataFrame()

    for idx, row in df.iterrows():
        comment = reddit.comment(id=row.id)
        comments_df['updated_score'] = comment.score
        comments_df['saved'] = comment.saved
        comments_df['updated_stickied'] = comment.stickied
        comments_df['num_replies'] = len(list(comment.replies))

        submission = reddit.submission(row.link_id[3:])
        # if line above doens't work, use line below
        # submission = reddit.submission(reddit.comment(id=row.id).submission.id)
        submissions_df = submissions_df.append({
            'id': submission.id,
            'name' : submission.name,
            'title' : submission.title,
            'num_comments' : submission.num_comments,
            'author' : submission.author,
            'created_utc' : datetime.fromtimestamp(submission.created_utc),
            'distinguished' : submission.distinguished,
            'is_self' : submission.is_self,
            'link_flair_text' : submission.link_flair_text,
            'locked' : submission.locked,
            'over_18' : submission.over_18,
            'permalink' : submission.permalink,
            'saved' : submission.saved,
            'score' : submission.score,
            'selftext' : submission.selftext,
            'spoiler' : submission.spoiler,
            'stickied' : submission.stickied,
            'upvote_ratio' : submission.upvote_ratio,
            'url' : submission.url
        }, ignore_index=True)

        user = reddit.redditor(row.author)
        users_df = users_df.append({
            'id': user.id,
            'comment_karma': user.comment_karma,
            'created_utc': datetime.fromtimestamp(user.created_utc),
            'has_verified_email': user.has_verified_email,
            'is_employee': user.is_employee,
            'is_mod': user.is_mod,
            'is_gold': user.is_gold,
            'link_karma': user.link_karma,
            'subreddit': user.subreddit
        }, ignore_index=True)
        
        return comments_df, submissions_df, users_df
        

In [381]:
# if we decide to get all comments for some posts:
# below is an instance of CommentForest
# print(reddit.submission(id=reddit.comment(id=df.id[0]).submission.id).comments)

In [382]:
# # if we decide to get all the comments from a user:
# user.comments
# user_subreddits = []
# for comment in reddit.redditor(user.name).comments.new(limit=None):
#     user_subreddits.append(comment.subreddit)
# 
# Counter(user_subreddits)
# print(len(user_subreddits)) #999, I wonder if it's cutting it off at 999...